## Programming: RNA Structure Prediction

In [8]:
import numpy as np
import re

In [9]:
def nussinov_fill(rna):
    """
    :param rna: the rna to build nussinovs matrix for
    :return: nm - nussinov's DP matrix
    Reference: https://www.cambridge.org/core/books/problems-and-solutions-in-biological-sequence-analysis/38C7399A53970CF4B4499EA036074F35 page 297
    """
    #dynamic programming matrix
    nm = np.empty([len(rna), len(rna)])
    nm[:] = np.NAN

    tie = False

    minimal_loop_length = 3

    # initialize diagonals of matrix
    for i in range(1, len(rna)):
        nm[i][i - 1] = 0
    for k in range(0, minimal_loop_length + 1):
        for i in range(0, len(rna) - k):
            nm[i][i + k] = 0

    #fill the matrix
    for offset in range(minimal_loop_length + 1, len(rna)):
        for col in range(len(rna) - 1, minimal_loop_length, -1):
            if col >= offset:
                row = col - offset
                down = nm[row + 1][col] #residue at rna[row] is hanging off by itself
                left = nm[row][col - 1] #residue at rna[col] is hanging off by itself
                diag = nm[row + 1][col - 1] + score((rna[row], rna[col])) #residue at rna[row] and rna[col] are paired if they are complementary
                rc = min([nm[row][k] + nm[k + 1][col] for k in range(row , col)])
                nm[row][col] = min([down, left, diag, rc])
                tie = min_tie([down, left, diag, rc])
    nm = np.around(nm, 1)
    return nm, tie

def min_tie(arr):
    current_min = np.Inf
    more_than_one_min = "NO"
    for elem in arr:
        if elem < current_min:
            current_min = elem
        elif elem == current_min:
            more_than_one_min = "YES"
        else:
            continue
    return more_than_one_min

def score(pair):
    scores = {('A','U'): -2,
              ('U','A'): -2,
              ('G','U'): -2,
              ('U','G'): -2,
              ('G','C'): -3,
              ('C','G'): -3}
    if pair in scores:
        return scores[pair]
    else:
        return 0

In [10]:
def nussinov_traceback(rna, nm):
    """
    :param rna: the rna sequence
    :param nm: the corresponding nussinov DP matrix for :param rna
    :return: the path through :param nm
    Reference: https://www.cambridge.org/core/books/problems-and-solutions-in-biological-sequence-analysis/38C7399A53970CF4B4499EA036074F35 page 296
    """
    i = 0
    j = len(rna) - 1
    stack = []
    stack.append((i , j))
    pair = []
    
    while stack: # is not empty
        tup_of_indices = stack.pop()
        i,j = tup_of_indices[0], tup_of_indices[1]
        if i >= j:
            continue
        elif nm[i][j - 1] == nm[i][j]:
            stack.append((i , j - 1))
        elif nm[i + 1][j] == nm[i][j]:
            stack.append((i + 1 , j))
        elif nm[i + 1][j - 1] + score((rna[i], rna[j])) == nm[i][j]:
            pair.append((i,j))
            stack.append((i + 1 , j - 1))
        else:
            for k in range(i + 1, j):
                if nm[i][k] + nm[k + 1][j] == nm[i][j]:
                    stack.append((k + 1, j))
                    stack.append((i, k))
                    break
    return pair

def nussinov_traceback_alt(rna, nm):
    """
    An alternate version of nussinov_traceback that has a different control flow through the while loop, this results in finding a different path through the matrix
    :param rna: the rna sequence
    :param nm: the corresponding nussinov DP matrix for :param rna
    :return: the path through :param nm
    Reference: https://www.cambridge.org/core/books/problems-and-solutions-in-biological-sequence-analysis/38C7399A53970CF4B4499EA036074F35 page 296
    """
    i = 0
    j = len(rna) - 1
    stack = []
    stack.append((i , j))
    pair = []
    
    while stack: # is not empty
        tup_of_indices = stack.pop()
        i,j = tup_of_indices[0], tup_of_indices[1]
        if i >= j:
            continue
        elif nm[i + 1][j] == nm[i][j]:
            stack.append((i + 1 , j))   
        elif nm[i][j - 1] == nm[i][j]:
            stack.append((i , j - 1))
        elif nm[i + 1][j - 1] + score((rna[i], rna[j])) == nm[i][j]:
            pair.append((i,j))
            stack.append((i + 1 , j - 1))
        else:
            for k in range(i + 1, j):
                if nm[i][k] + nm[k + 1][j] == nm[i][j]:
                    stack.append((k + 1, j))
                    stack.append((i, k))
                    break
    return pair

def traceback_all(rna, nm):
    list_of_pairs = []
    list_of_pairs.append(nussinov_traceback(rna, nm))
    list_of_pairs.append(nussinov_traceback_alt(rna, nm))
    return list_of_pairs

def dot_write(rna, fold):
    """
    :param rna: the rna sequence to find the dot bracket structure for
    :param fold: the indices of rna where a the loop starts and ends
    :return: the dot bracket structure for :param rna
    Reference: https://bayesianneuron.com/2019/02/nussinov-predict-2nd-rna-fold-structure-algorithm/
    """
    dot = ["." for i in range(len(rna))]
    for s in fold:
        dot[min(s)] = "("
        dot[max(s)] = ")"
    return "".join(dot)

def get_mfe(dot, rna, nm):
    """
    :param dot: the dot bracket structure for the input rna
    :param rna: the rna sequence
    :param nm: the corresponding nussinov DP matrix for :param rna
    :return: the mfe of the dot bracket structure with unpaired loop bases accounted for
    """
    mfe = nm[0][len(rna) - 1]
    unpaired = re.findall(r'\(.*?\)', dot)
    for elem in unpaired:
        for char in elem:
            if char == ".":
                mfe = mfe + 0.1
    return mfe.round(1)

## Handle the file I/O

In [11]:
with open('5.in', 'r') as f:
    lines = f.readlines()
    if len(lines) == 1:
        seq = lines[0].upper()
    elif len(lines) == 2:
        seq = lines[1].upper()
    else:
        print("Error: Input file must have exactly one or two lines")

## (a)

In [12]:
# set up everything we need to print the results
matrix = nussinov_fill(seq)[0]
traceback = nussinov_traceback(seq, matrix)
structure = dot_write(seq, traceback)
energy = get_mfe(structure, seq, matrix)

#output results
print(seq)
print(energy)
print(structure)

GGCACUGAA
-2.7
(...)....


## (b)

In [13]:
tie = nussinov_fill(seq)[1]
print(tie)

YES


## (c)

In [14]:
all_tracebacks = traceback_all(seq, matrix)
print(len(all_tracebacks))
print(dot_write(seq, all_tracebacks[0]))
print(dot_write(seq, all_tracebacks[1]))

2
(...)....
..(...)..


## Recursion comprehension
![](V-recursion.png)

Shown above is the recursion for the V matrix of Zuker's algorithm.  
Tan *et al.* give an alternate notation for the recursion (shown below) which will be used for the remainder of this explanation.  
  
![](V-alternate.png)

In the above alternate recursion for V, we can see four terms:
* $eH(i, j)$ is the energy of a hairpin loop
* $eS(i, j)$ is the energy of stacking base pair $(i, j)$ with $(i + 1, j - 1)$
* $VBI(i, j)$ is the energy of an internal loop closed by base pair $(i, j)$
* $VM(i ,j)$ is the energy of a multi-branched loop closed by base pair $(i, j)$
  
In order to understand why Zuker's algorithm is $O(n^4)$ we can must look more closely at how $VBI(i, j)$ is calculated according to Tan *et al.* (shown below)  
Note: $VBI(i, j)$ in the recursion given by Tan *et al.* corresponds to the $min_{\,r,r'}e_{\,int}(i,r,r',j')+V_{\,r,r'}$ term of the recursion given in the assignment pdf

![](VBI.png)  
  
As we can see, we must calculate $VBI(i',j')$ for the conditions $i < i' < j < j'$ and $i' - i + j - j' > 2$ each time we need to calculate $VBI(i,j)$.  
Therefore, we need to iterate through $(i',j')$ and $(i,j)$ for each calculation of $VBI(i,j)$, this requires $O(n^4)$ time and gives Zuker's algorithm an overall time of $O(n^4)$ as it is the most expensive step.

In order to reduce this time to $O(n^3)$ Tan *et al.* devise an optimization which is requires extra $O(n)$ memory.    
The optimization involves storing the energy for any internal loop which is calculated, then when a loop of same size appears instead of redundantly calculating this energy the algorithm can just look up this value with a **presumably** $O(1)$ look up time.  

Thus, the optimzation presented by Tan *et al* reduces the time complexity of Zuker's algorithm to $O(^3)$

Reference:  
Tan, G., Liu, X. and Sun, N., 2005, May. An efficient dynamic programming algorithm and implementation for RNA secondary structure prediction. In International Conference on Computational Science (pp. 869-876). Springer, Berlin, Heidelberg.

## Covariance Model  
## (a)  
The parse tree for the SCFG is shown below. This is based on figure 9.5 of the textbook.  
  
![](parse_tree.png)  

## (b)  
The covariance model shown below, based on figure 10.12 of the textbook.  
The left side of the red line shows the general model while the right side shows the model after being used to parse the consensus sequence.  
    
![](SCFG-model.png)  

## (c)  
The covariance model should accept all three of the sequences shown in the assignment 5 pdf.  
This is because the covariance model is based off of the consensus sequence for the three sequences outlined in the pdf.  

## (d)  
In order to expand the covariance model to accept any sequence with the structure $(..(((.....))((...)).).)$ we would need to change the model as shown below.  
The changes are shown in green.  
  
![](modified-model.png)